<a href="https://colab.research.google.com/github/hoa92ng/HashTation/blob/main/crawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing import List, Tuple
from datetime import datetime
import os
import sys
import time
import cv2
import pyautogui

def parse_account(file_path: str = "./acc.txt") -> List[Tuple[str, str]]:
    accounts = []
    # Kiểm tra xem tệp có tồn tại hay không
    if not os.path.exists(file_path):
        print(f"Error: File '{file_path}' does not exist!")
        time.sleep(5)
        sys.exit()
    # Read all lines from the file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Parse each line
    for line in lines:
        line = line.strip()  # Remove whitespace and newline characters
        if not line:  # Skip empty lines
            continue

        parts = line.split(':')  # Split the line by ':'
        if len(parts) == 2:
            username, password = parts[0], parts[1]
            accounts.append((username, password))

    return accounts

def parse_proxy(file_path: str = "./proxy.txt") -> List[Tuple[str, int, str, str]]:
    proxies = []

    if not os.path.exists(file_path):
        print(f"Error: File '{file_path}' does not exist!")
        time.sleep(5)
        sys.exit()

    # Read all lines from the file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Parse each line
    for line in lines:
        line = line.strip()  # Remove whitespace and newline characters
        if not line:  # Skip empty lines
            continue

        parts = line.split(':')  # Split the line by ':'
        if len(parts) == 4:
            ip = parts[0]
            port = int(parts[1])  # Convert the port to an integer
            username = parts[2]
            password = parts[3]

            proxies.append((ip, port, username, password))
        elif len(parts) == 2:
            ip = parts[0]
            port = int(parts[1])  # Convert the port to an integer

            proxies.append((ip, port))

    return proxies

def log(log, idx, tqdm=None):
    _date_now = datetime.now().now().strftime("%Y-%m-%d %H:%M:%S")
    _log = "[{0}] [Thread {1}] {2}".format(_date_now, idx, log)
    if tqdm:
        tqdm.write(_log)
    else:
        print(_log)


# Tìm vị trí của hình ảnh trong màn hình
def find_image_on_screen(screen, template, threshold=0.8):
    # So khớp ảnh bằng phương pháp template matching
    result = cv2.matchTemplate(screen, template, cv2.TM_CCOEFF_NORMED)

    # Tìm vị trí có độ khớp cao hơn ngưỡng (threshold)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    if max_val >= threshold:
        top_left = max_loc
        h, w = template.shape[:2]
        bottom_right = (top_left[0] + w, top_left[1] + h)
        return {"top_left": top_left, "bottom_right": bottom_right, "confidence": max_val, "width": w, "height": h}
    else:
        return None

def move_mouse_to_position(x, y):

    # Di chuyển chuột đến vị trí trung tâm
    pyautogui.moveTo(x, y, duration=0.5)  # Thời gian di chuyển 0.5 giây
    pyautogui.click()  # Click chuột trái
    # print(f"Chuột đã di chuyển đến tọa độ: ({center_x}, {center_y})")

In [ ]:
from utils import *
from webs import Auto_Click
import threading
from time import sleep
import tqdm

time = 1
try:
    user_input = input("Input cycle time (in mins. Default: 1): ")
    if not user_input.strip():
        time = 1
    else:
        time = int(user_input)
except ValueError:
    time = 1
COUNT_DOWN = time * 60
count = COUNT_DOWN
progress_bar = tqdm(range(COUNT_DOWN), desc="Time")
proxies = parse_proxy()
account = parse_account()
auto_click_instances = []

# create instance
for i, proxy in enumerate(proxies):
            # thread = threading.Thread(target=)
    url = "https://freebitco.in/"
    sub_url = "https://freebitco.in/?op=home"
    auto_click = Auto_Click(url=url,
                            sub_url=sub_url,
                            reconnect_time=3,
                            in_proxy=proxy,
                            account=account[0],
                            count_down=time*60,
                            thread_no=i)
    auto_click_instances.append(auto_click)

while(True):
    tqdm.update(1)
    if count == 0:
        tqdm.reset()
        count = COUNT_DOWN
        for i, auto_click_instance in enumerate(auto_click_instances):
             auto_click_instance.run()
    sleep(1)
    time -= 1

In [ ]:
from seleniumbase import Driver
from time import sleep
from utils import log, find_image_on_screen, move_mouse_to_position
import threading
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import cv2
import pyautogui

class Auto_Click:
    def __init__(self, url, sub_url, reconnect_time, in_proxy, account, count_down, thread_no):
        self.url = url
        self.sub_url = sub_url
        self.reconnect_time = reconnect_time
        self.proxy = in_proxy
        self.account = account
        self.driver = None
        self.COUNT_DOWN = count_down
        self.tqdm = tqdm(range(self.COUNT_DOWN), desc=f"Thread {thread_no}")
        self.thread_no = thread_no
        self.uc_open_with_reconnect()
        self.thread = threading.Thread(target=self.run)
        self.thread.start()
        self.screenshot_path = "./screenshot.png"
        self.roll_img = cv2.imread("./Roll.png", cv2.IMREAD_COLOR_BGR)
        self.verify_img = cv2.imread("./verify.png", cv2.IMREAD_COLOR_BGR)
        self.success_img = cv2.imread("./Success.png", cv2.IMREAD_COLOR_BGR)
        self.insert_data()

    def capture_screenshot(self):
        screenshot = pyautogui.screenshot()  # Chụp toàn bộ màn hình
        screenshot.save(self.screenshot_path)
        return cv2.imread(self.screenshot_path, cv2.IMREAD_COLOR_BGR)

    def uc_open_with_reconnect(self):
        # Prepare proxy configuration
        if self.proxy:
            proxy = None
            if len(self.proxy) == 4:
                proxy = f"{self.proxy[2]}:{self.proxy[3]}@{self.proxy[0]}:{self.proxy[1]}"
                log(f"Using proxy {self.proxy[0]}:{self.proxy[1]}:{self.proxy[2]}:{self.proxy[3]} to access", self.thread_no, self.tqdm)
            else:
                proxy = f"{self.proxy[0]}:{self.proxy[1]}"
                log(f"Using proxy {self.proxy[0]}:{self.proxy[1]} to access", self.thread_no, self.tqdm)

            self.driver = Driver(
            uc=True,
            headless=False,
            proxy=proxy  # Add the proxy here
            )
        else:
            self.driver = Driver(
            uc=True,
            headless=False,
            )

        # Open the URL
        self.driver.maximize_window()
        self.driver.uc_open_with_reconnect(self.url, reconnect_time=self.reconnect_time)
        sleep(2)


    def insert_data(self):
        try:
            # Tìm button bằng ID
            button = self.driver.find_element(By.CLASS_NAME, "login_menu_button")
            button.click()
            log("Button 'Login' clicked successfully!", self.thread_no, self.tqdm)
        except Exception as e:
            # log(f"Error: When finding login button ({str(e)})", self.thread_no, self.tqdm)
            log(f"Error: When finding login button", self.thread_no, self.tqdm)
        sleep(2)
        try:
            # Tìm button bằng ID
            user = self.driver.find_element("id", "login_form_btc_address")
            user.clear()
            # Enter text into the textbox
            user.send_keys(self.account[0])
            log(f"Inputed user: {self.account[0]}", self.thread_no, self.tqdm)
        except Exception as e:
            # log(f"Error: When inputing user button ({str(e)})", self.thread_no, self.tqdm)
            log(f"Error: When inputing user button", self.thread_no, self.tqdm)
        sleep(2)
        try:
            # Tìm button bằng ID
            password = self.driver.find_element("id", "login_form_password")
            password.clear()
            # Enter text into the textbox
            password.send_keys(self.account[1])
            log(f"Inputed password: *************", self.thread_no, self.tqdm)
        except Exception as e:
            # log(f"Error: When inputing password button ({str(e)})", self.thread_no, self.tqdm)
            log(f"Error: When inputing password button", self.thread_no, self.tqdm)
        sleep(2)

    def run(self):
        try:
            # Tìm button bằng ID
            self.driver.switch_to.window(self.driver.current_window_handle)
            roll = self.driver.find_element("id", "free_play_form_button")
            screen = self.capture_screenshot()
            log("1st Captured screen", self.thread_no, self.tqdm)
            coordinate = find_image_on_screen(screen=screen, template=self.verify_img)
            if coordinate:
                move_mouse_to_position(coordinate['top_left'][0] + 40, coordinate['top_left'][1] + coordinate['height'] / 2)
                sleep(20)
                log("1st Clicked on verify button", self.thread_no, self.tqdm)
                screen = self.capture_screenshot()
                log("2nd Captured screen", self.thread_no, self.tqdm)
                coordinate = find_image_on_screen(screen=screen, template=self.success_img)
                if not coordinate:
                    self.driver.execute_script("window.open('about:blank', '_blank');")
                    sleep(1)
                    self.driver.switch_to.window(self.driver.window_handles[-1])
                    sleep(1)
                    log("Opened a new tab", self.thread_no, self.tqdm)
                    self.driver.switch_to.window(self.driver.window_handles[0])
                    sleep(1)
                    self.driver.execute_script("window.close();")  # Close it using JavaScript
                    log("Closed the original tab", self.thread_no, self.tqdm)
                    sleep(2)
                    self.driver.switch_to.window(self.driver.window_handles[-1])
                    sleep(1)
                    self.driver.uc_open_with_reconnect(self.sub_url, reconnect_time=self.reconnect_time)
                    sleep(5)
                    self.driver.switch_to.window(self.driver.window_handles[-1])  # Switch to the new tab
                    sleep(2)
                    screen = self.capture_screenshot()
                    log("3th Captured screen", self.thread_no, self.tqdm)
                    coordinate = find_image_on_screen(screen=screen, template=self.verify_img)
                    if coordinate:
                        move_mouse_to_position(coordinate['top_left'][0] + 40, coordinate['top_left'][1] + coordinate['height'] / 2)
                        sleep(20)
                        log("2nd Clicked on verify button", self.thread_no, self.tqdm)
                        screen = self.capture_screenshot()
                        log("4nd Captured screen", self.thread_no, self.tqdm)
                        coordinate = find_image_on_screen(screen=screen, template=self.success_img)
                        if coordinate:
                            roll = self.driver.find_element("id", "free_play_form_button")
                            sleep(2)
                            roll.click()
                            log("Clicked Roll button", self.thread_no, self.tqdm)
                            sleep(15)
                            close = self.driver.find_elements(By.CLASS_NAME, "close-reveal-modal")
                            if len(close) > 0:
                                log("Clicked closing button", self.thread_no, self.tqdm)
                                close[0].click()
                else:
                    roll = self.driver.find_element("id", "free_play_form_button")
                    sleep(2)
                    roll.click()
                    log("Clicked Roll button", self.thread_no, self.tqdm)
                    sleep(15)
                    close = self.driver.find_elements(By.CLASS_NAME, "close-reveal-modal")
                    if len(close) > 0:
                        log("Clicked closing button", self.thread_no, self.tqdm)
                        close[0].click()
        except Exception as e:
            log(f"Error: When search Roll button ({str(e)})", self.thread_no, self.tqdm)
            # log(f"Error: When search Roll button", self.thread_no, self.tqdm)